# Experiments

### Setup

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv()

True

Here is the RAG Application that we've been working with throughout this course

In [9]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
# from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langsmith import traceable
# from openai import OpenAI
from anthropic import Anthropic
from typing import List
import nest_asyncio

# TODO: Configure this model!
# MODEL_NAME = "claude-3-5-haiku-latest"
MODEL_NAME = "claude-sonnet-4-5"
MODEL_PROVIDER = "claude"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

anthropic_client = Anthropic()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = HuggingFaceEmbeddings(
        model_name="all-MiniLM-l6-v2",
        model_kwargs={'device': 'mps'},
        encode_kwargs={'normalize_embeddings': False}
    )

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_claide` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_claude(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_claude(messages: List[dict]) -> str:
    return anthropic_client.messages.create(
        model=MODEL_NAME,
        messages=messages,
        system=RAG_SYSTEM_PROMPT,
        temperature=0,
        max_tokens=1024
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.content[0].text


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [7]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "golden-dataset-module2"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix=MODEL_NAME
)

View the evaluation results for experiment: 'claude-3-5-haiku-latest-2e00f52d' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/20db0460-c1aa-4e70-ae79-6b66939d57c2/compare?selectedSessions=883b51a0-6d37-40c5-b14f-4194391b5ab6




12it [00:35,  2.94s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can I use langsmith with claude?,"I apologize, but I cannot find specific inform...",None,"Yes, you can use langsmith with claude.",0,2.779284,371ddbe9-ee75-4d1a-85a5-78a28d18da20,4a2c6d57-8117-4f37-a76c-5e39357479c6
1,How do I set up tracing to LangSmith if I'm us...,"I apologize, but while the context mentions La...",None,"I apologize, but while the context mentions La...",1,3.124947,9c6224dd-0516-446c-8b45-588bc3957d80,7cc52ef1-e71d-4878-9e66-cb8caa01119b
2,Can LangSmith be used for finetuning and model...,"I apologize, but I cannot find specific inform...",None,"Yes, LangSmith can be used for fine-tuning and...",1,2.552456,438c9b04-0992-4a78-bf5c-7d36cf0dae4b,fea0b3de-1923-4b67-89e9-1880285dec92
3,Does LangSmith support offline evaluation?,"I apologize, but I cannot find any information...",None,"Yes, LangSmith supports offline evaluation thr...",1,2.876576,477928c1-385a-458c-ac56-ce918db23cde,9960e218-4961-401e-b86e-7db2592d1b28
4,How can I trace with the @traceable decorator?,"I apologize, but I cannot find information abo...",None,To trace with the @traceable decorator in Pyth...,1,2.881547,4b0c0d94-6306-409b-8d1f-9831cfa9d8d8,fd8ed9b8-3a94-4976-955c-5ce279838bbc
5,What testing capabilities does LangSmith have?,"I apologize, but I cannot find any information...",None,LangSmith offers capabilities for creating dat...,1,2.869535,4ec23ef9-b83c-45ab-a3f2-d9ed4e028d93,9e269d5e-c201-473a-8828-306a19af843d
6,How do I create user feedback with the LangSmi...,"I apologize, but I cannot find specific inform...",None,To create user feedback with the LangSmith SDK...,1,2.965943,71d5f4ed-e02f-4d48-b14c-bcabea990f11,4c602945-e159-4256-99db-95b2804cc5ea
7,How do I pass metadata in with @traceable?,"I apologize, but I cannot find information in ...",None,You can pass metadata with the @traceable deco...,1,2.953210,77cf8ea3-5ffd-48f1-9e5f-9f5e503050bd,29a7ab56-f040-4071-b2ad-bab7363cf5f9
8,What is LangSmith used for in three sentences?,"I apologize, but I cannot find any information...",None,LangSmith is a platform designed for the devel...,1,2.582647,a169ea72-e006-4355-b47c-aa2ebf525d71,92f043b1-8777-47da-b60c-e90bef217435
9,Does LangSmith support online evaluation?,"I apologize, but I cannot find any information...",None,"Yes, LangSmith supports online evaluation as a...",1,3.042015,b0e71a43-204e-4cc3-acd2-2f8e2603cf62,ca5f0ae4-9a28-4994-9c29-7d0117df07db


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [10]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="claude-sonnet-4-5"
)

View the evaluation results for experiment: 'claude-sonnet-4-5-fa6fcdbc' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/20db0460-c1aa-4e70-ae79-6b66939d57c2/compare?selectedSessions=f3b2d096-2aba-4037-b927-0b716a163e6e




12it [00:51,  4.32s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can I use langsmith with claude?,I don't see any information in the provided co...,None,"Yes, you can use langsmith with claude.",0,3.510754,371ddbe9-ee75-4d1a-85a5-78a28d18da20,322e637a-e6ed-43c7-b484-ca28a9813116
1,How do I set up tracing to LangSmith if I'm us...,I don't see any information about setting up t...,None,"I apologize, but while the context mentions La...",0,3.884151,9c6224dd-0516-446c-8b45-588bc3957d80,0cda7fd3-c803-4a47-bd0b-34c1f08f2ab5
2,Can LangSmith be used for finetuning and model...,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith can be used for fine-tuning and...",0,4.089769,438c9b04-0992-4a78-bf5c-7d36cf0dae4b,b3b3d722-ab6f-427d-b333-f5de5a76856d
3,Does LangSmith support offline evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports offline evaluation thr...",0,4.099601,477928c1-385a-458c-ac56-ce918db23cde,0b0b5bae-908e-45ba-84f2-4a358c0a3f65
4,How can I trace with the @traceable decorator?,"Based on the provided context, I don't see any...",None,To trace with the @traceable decorator in Pyth...,1,7.199192,4b0c0d94-6306-409b-8d1f-9831cfa9d8d8,8f06f13b-4fa6-4c0d-a552-5d462addb894
5,What testing capabilities does LangSmith have?,I don't have information about LangSmith's tes...,None,LangSmith offers capabilities for creating dat...,1,3.604573,4ec23ef9-b83c-45ab-a3f2-d9ed4e028d93,1d159e19-ae7a-411c-9ca0-9c0f61e0c9da
6,How do I create user feedback with the LangSmi...,I don't see any information about creating use...,None,To create user feedback with the LangSmith SDK...,1,4.114326,71d5f4ed-e02f-4d48-b14c-bcabea990f11,022532d7-51eb-421d-b1fa-aa9005a276d6
7,How do I pass metadata in with @traceable?,I don't see any information about `@traceable`...,None,You can pass metadata with the @traceable deco...,0,5.326375,77cf8ea3-5ffd-48f1-9e5f-9f5e503050bd,d77b354e-d730-438c-b168-13656236d49a
8,What is LangSmith used for in three sentences?,I don't know what LangSmith is used for based ...,None,LangSmith is a platform designed for the devel...,1,3.953612,a169ea72-e006-4355-b47c-aa2ebf525d71,99cb6605-ef85-4efb-adbc-f417617347b0
9,Does LangSmith support online evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports online evaluation as a...",0,3.779360,b0e71a43-204e-4cc3-acd2-2f8e2603cf62,0aeb9819-916e-4bf9-af23-949359a00b52


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [13]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="v0 initial"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="v0 initial"
)

View the evaluation results for experiment: 'v0 initial-030e1516' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/20db0460-c1aa-4e70-ae79-6b66939d57c2/compare?selectedSessions=fc21540c-fb74-4ed7-87c4-ab78deb6629b




10it [00:40,  4.10s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used for finetuning and model...,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith can be used for fine-tuning and...",0,4.588703,438c9b04-0992-4a78-bf5c-7d36cf0dae4b,7c16b18b-a5af-4546-92b3-9a272252b6cc
1,Does LangSmith support offline evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports offline evaluation thr...",0,4.525131,477928c1-385a-458c-ac56-ce918db23cde,6d75ed41-58e0-48e6-b3a1-07e738a69e60
2,How can I trace with the @traceable decorator?,"Based on the provided context, I don't see any...",None,To trace with the @traceable decorator in Pyth...,1,4.422527,4b0c0d94-6306-409b-8d1f-9831cfa9d8d8,e13b04e1-d575-41db-ba51-52e297d3ff75
3,What testing capabilities does LangSmith have?,I don't have information about LangSmith's tes...,None,LangSmith offers capabilities for creating dat...,1,3.371892,4ec23ef9-b83c-45ab-a3f2-d9ed4e028d93,c98fc43f-e68d-41d2-8cb4-20f930e682d2
4,How do I create user feedback with the LangSmi...,I don't see any information about creating use...,None,To create user feedback with the LangSmith SDK...,1,3.518390,71d5f4ed-e02f-4d48-b14c-bcabea990f11,7578e5b7-65d1-443d-89ca-f88cdaf9f571
5,How do I pass metadata in with @traceable?,I don't see any information about `@traceable`...,None,You can pass metadata with the @traceable deco...,0,4.847408,77cf8ea3-5ffd-48f1-9e5f-9f5e503050bd,c552fda5-74be-45f6-82d3-9f4d7d2fd17b
6,What is LangSmith used for in three sentences?,I don't know what LangSmith is used for based ...,None,LangSmith is a platform designed for the devel...,1,3.379950,a169ea72-e006-4355-b47c-aa2ebf525d71,df76f36a-b2be-4bc6-b5be-ceddd656b7a2
7,Does LangSmith support online evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports online evaluation as a...",0,3.892389,b0e71a43-204e-4cc3-acd2-2f8e2603cf62,f8a1eb5e-6bad-4f30-ab16-9755a3577955
8,Can LangSmith be used to evaluate agents?,I don't have information about LangSmith in th...,None,"Yes, LangSmith can be used to evaluate agents....",1,3.463698,c8addc8f-e8e4-4d19-9662-4bbd4ff3a753,916642af-ce17-4052-a693-e534d6da7ea5
9,How do I set up tracing to LangSmith if I'm us...,I don't see any information about setting up t...,None,To set up tracing to LangSmith while using Lan...,0,4.077732,f5c6825c-4d90-4ddb-bae0-f54ce17872ce,4382181d-24e3-4ed3-95ff-7a69ed6ccc36


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [15]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["new_data"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="new_data"
)

View the evaluation results for experiment: 'new_data-ec807d7e' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/20db0460-c1aa-4e70-ae79-6b66939d57c2/compare?selectedSessions=74702af3-9bc5-4711-aebd-ad3a71c263ed




2it [00:07,  3.95s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can I use langsmith with claude?,I don't see any information in the provided co...,None,"Yes, you can use langsmith with claude.",0,3.602624,371ddbe9-ee75-4d1a-85a5-78a28d18da20,de404bc0-bec5-4d76-be12-0dbaf574517e
1,How do I set up tracing to LangSmith if I'm us...,I don't see any information about setting up t...,None,"I apologize, but while the context mentions La...",0,3.809085,9c6224dd-0516-446c-8b45-588bc3957d80,3a9a559e-9c27-40ab-888f-04a741de008e


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [16]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            "438c9b04-0992-4a78-bf5c-7d36cf0dae4b",
            "f5c6825c-4d90-4ddb-bae0-f54ce17872ce"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-c4857df2' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/20db0460-c1aa-4e70-ae79-6b66939d57c2/compare?selectedSessions=3cc3d275-5634-4456-9ce0-cfc3f0041fda




2it [00:09,  4.51s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used for finetuning and model...,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith can be used for fine-tuning and...",0,4.105664,438c9b04-0992-4a78-bf5c-7d36cf0dae4b,5fcb34c7-6db2-42f5-85f8-663920603c61
1,How do I set up tracing to LangSmith if I'm us...,I don't see any information about setting up t...,None,To set up tracing to LangSmith while using Lan...,0,4.409966,f5c6825c-4d90-4ddb-bae0-f54ce17872ce,23ee2543-ccf5-479d-ab7e-d36ee35517af


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [17]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-ea30eb6f' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/20db0460-c1aa-4e70-ae79-6b66939d57c2/compare?selectedSessions=b61d3bc7-40eb-49e4-bef4-81c0e60d1884




24it [01:33,  3.88s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can I use langsmith with claude?,I don't see any information in the provided co...,None,"Yes, you can use langsmith with claude.",0,4.142295,371ddbe9-ee75-4d1a-85a5-78a28d18da20,4389e5a9-40b6-4dc0-8325-189ccfaa47b5
1,How do I set up tracing to LangSmith if I'm us...,I don't see any information about setting up t...,None,"I apologize, but while the context mentions La...",0,4.383652,9c6224dd-0516-446c-8b45-588bc3957d80,32fef202-bf62-4b01-a289-40f18a30424d
2,Can LangSmith be used for finetuning and model...,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith can be used for fine-tuning and...",0,3.900740,438c9b04-0992-4a78-bf5c-7d36cf0dae4b,0a6dd9ca-a57c-4ece-b9fe-fcff73ffa582
3,Does LangSmith support offline evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports offline evaluation thr...",0,4.028032,477928c1-385a-458c-ac56-ce918db23cde,d9077874-d8f7-4f67-b9ae-70438a5d7146
4,How can I trace with the @traceable decorator?,"Based on the provided context, I don't see any...",None,To trace with the @traceable decorator in Pyth...,1,3.988998,4b0c0d94-6306-409b-8d1f-9831cfa9d8d8,7be42bdc-8772-41d9-8fda-25c3664d45ce
5,What testing capabilities does LangSmith have?,I don't have information about LangSmith's tes...,None,LangSmith offers capabilities for creating dat...,1,3.598225,4ec23ef9-b83c-45ab-a3f2-d9ed4e028d93,2176a702-20ca-458e-90ec-d5cc5c58945a
6,How do I create user feedback with the LangSmi...,I don't see any information about creating use...,None,To create user feedback with the LangSmith SDK...,1,3.295173,71d5f4ed-e02f-4d48-b14c-bcabea990f11,becbe857-74d7-43a7-a4a2-95775837821c
7,How do I pass metadata in with @traceable?,I don't see any information about `@traceable`...,None,You can pass metadata with the @traceable deco...,0,4.118076,77cf8ea3-5ffd-48f1-9e5f-9f5e503050bd,2ed57fe6-8854-4758-92ba-7630670c9581
8,What is LangSmith used for in three sentences?,I don't know what LangSmith is used for based ...,None,LangSmith is a platform designed for the devel...,1,3.757066,a169ea72-e006-4355-b47c-aa2ebf525d71,936087a0-b4e4-4297-aace-ae5cc2bbe605
9,Does LangSmith support online evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports online evaluation as a...",0,4.084427,b0e71a43-204e-4cc3-acd2-2f8e2603cf62,feeb2144-b6f4-40dc-905d-4b74d89a72e7


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [18]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-dd3c1839' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/20db0460-c1aa-4e70-ae79-6b66939d57c2/compare?selectedSessions=65c3f236-4ddc-4e72-9814-d5db1fd5fb9a




12it [00:16,  1.36s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can I use langsmith with claude?,I don't see any information in the provided co...,None,"Yes, you can use langsmith with claude.",0,3.377388,371ddbe9-ee75-4d1a-85a5-78a28d18da20,8c81ad93-7fdb-4524-a2f5-c302cdb5b234
1,How do I set up tracing to LangSmith if I'm us...,I don't see any information about setting up t...,None,"I apologize, but while the context mentions La...",0,3.793513,9c6224dd-0516-446c-8b45-588bc3957d80,359049ee-785b-46f3-99f2-3be9833fda1e
2,Can LangSmith be used for finetuning and model...,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith can be used for fine-tuning and...",0,3.870380,438c9b04-0992-4a78-bf5c-7d36cf0dae4b,61fbe50f-f1ff-4eec-825a-432c8427aa14
3,What testing capabilities does LangSmith have?,I don't have information about LangSmith's tes...,None,LangSmith offers capabilities for creating dat...,1,3.166720,4ec23ef9-b83c-45ab-a3f2-d9ed4e028d93,b133e7e2-5af5-4271-897d-c965cf9d9214
4,Does LangSmith support offline evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports offline evaluation thr...",0,4.247964,477928c1-385a-458c-ac56-ce918db23cde,ddfec0c4-a186-49aa-8a8f-fa87adb9e0e8
5,How can I trace with the @traceable decorator?,"Based on the provided context, I don't see any...",None,To trace with the @traceable decorator in Pyth...,1,4.127409,4b0c0d94-6306-409b-8d1f-9831cfa9d8d8,3900350d-1a53-4b51-8e70-6a245ad29e8b
6,How do I create user feedback with the LangSmi...,I don't see any information about creating use...,None,To create user feedback with the LangSmith SDK...,1,3.886673,71d5f4ed-e02f-4d48-b14c-bcabea990f11,3de22411-1344-4ff1-8370-6e1e0f241be4
7,What is LangSmith used for in three sentences?,I don't know what LangSmith is used for based ...,None,LangSmith is a platform designed for the devel...,1,3.004574,a169ea72-e006-4355-b47c-aa2ebf525d71,00afb0f6-65e9-4808-a3a2-42c6ee7f1d72
8,How do I pass metadata in with @traceable?,I don't see any information about `@traceable`...,None,You can pass metadata with the @traceable deco...,0,4.323552,77cf8ea3-5ffd-48f1-9e5f-9f5e503050bd,97b5a499-4c53-479a-81ed-bf0b5e865a44
9,Can LangSmith be used to evaluate agents?,I don't have information about LangSmith in th...,None,"Yes, LangSmith can be used to evaluate agents....",1,3.682095,c8addc8f-e8e4-4d19-9662-4bbd4ff3a753,42f87f0c-f6fe-4ca2-9c7a-50bb6645766a


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [19]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-54a0cf12' at:
https://smith.langchain.com/o/e37fb4e5-4567-4839-b84a-654be8101cf2/datasets/20db0460-c1aa-4e70-ae79-6b66939d57c2/compare?selectedSessions=b18f23b3-957a-440e-9ce7-a4ab46464c58




12it [00:47,  3.95s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can I use langsmith with claude?,I don't see any information in the provided co...,None,"Yes, you can use langsmith with claude.",0,3.647831,371ddbe9-ee75-4d1a-85a5-78a28d18da20,dc1cc2ce-76a0-4a4e-948b-646763515bd1
1,How do I set up tracing to LangSmith if I'm us...,I don't see any information about setting up t...,None,"I apologize, but while the context mentions La...",0,3.813013,9c6224dd-0516-446c-8b45-588bc3957d80,65b796f2-6c1c-4ff4-be43-be38e07a55f7
2,Can LangSmith be used for finetuning and model...,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith can be used for fine-tuning and...",0,4.615943,438c9b04-0992-4a78-bf5c-7d36cf0dae4b,91ef147c-340c-4502-8f1f-e7722fa5490d
3,Does LangSmith support offline evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports offline evaluation thr...",0,4.242927,477928c1-385a-458c-ac56-ce918db23cde,14cb2781-c903-4129-9ac0-bc52fd30e9b1
4,How can I trace with the @traceable decorator?,"Based on the provided context, I don't see any...",None,To trace with the @traceable decorator in Pyth...,1,3.987689,4b0c0d94-6306-409b-8d1f-9831cfa9d8d8,4395799a-ffde-4c7a-b83d-9299b11c905f
5,What testing capabilities does LangSmith have?,I don't have information about LangSmith's tes...,None,LangSmith offers capabilities for creating dat...,1,3.677282,4ec23ef9-b83c-45ab-a3f2-d9ed4e028d93,2039140d-253a-4257-b647-0ecd35b8ae3a
6,How do I create user feedback with the LangSmi...,I don't see any information about creating use...,None,To create user feedback with the LangSmith SDK...,1,3.175820,71d5f4ed-e02f-4d48-b14c-bcabea990f11,07824a71-c9fb-4be9-85a6-e6923a112972
7,How do I pass metadata in with @traceable?,I don't see any information about `@traceable`...,None,You can pass metadata with the @traceable deco...,0,4.412253,77cf8ea3-5ffd-48f1-9e5f-9f5e503050bd,54b9c75e-c273-4a60-8458-391f4a102ae5
8,What is LangSmith used for in three sentences?,I don't know what LangSmith is used for based ...,None,LangSmith is a platform designed for the devel...,1,3.988954,a169ea72-e006-4355-b47c-aa2ebf525d71,f1d472df-fbba-4638-936c-49c59b771cc5
9,Does LangSmith support online evaluation?,"Based on the provided context, I cannot find a...",None,"Yes, LangSmith supports online evaluation as a...",0,3.675646,b0e71a43-204e-4cc3-acd2-2f8e2603cf62,ece98deb-307a-4e41-9e5e-a26abb4623db
